# SCRIPT DESCRIPTION

# IMPORTING LIBRARIES

In [1]:
# Please check to ensure that all of the libraries imported in functions_curtailing_bci2000_data.py are installed
# in your environment or in the same file pathway. 
import functions_curtailing_bci2000_data


# INPUTTING PARAMETERS

In [2]:
"""
DESCRIPTION:
Please input the parameters below.
"""

# EXPERIMENTER INPUTS:
block_id          = 'Block3'
date              = '2023_01_05'
dir_bci2000_data  = '/mnt/shared/ecog/'
dir_intermediates = '/mnt/shared/danprocessing/Projects/PseudoOnlineTests_for_RTCoG/Intermediates/'
patient_id        = 'CC01'
task              = 'Speller'

"""
INPUT VARIABLES:
block_id:          [String (BlockX, where X is an int))]; Block ID of the task that was run.
date:              [string (YYYY_MM_DD)]; Date on which the block was run.
dir_bci2000_data:  [string]; Directory where the BCI2000 data is stored.
dir_intermediates: [string]; Intermediates directory where relevant information is stored.
patient_id:        [string]; Patient ID PYyyNnn or CCxx format, where y, n, and x are integers.
task:              [string]; Type of task that was run.
""";


# LOADING .MAT FILE

In [3]:
"""
DESCRIPTION:
Loading the matlab file whose data (signals and states) we will align and crop. The .mat file 
is imported from:

dir_bci2000_data + patient_id + '/mat/' + date + '/' + task + '_' + block_id + '.mat'

Feel free to modify the pathway in which this .mat file is stored and the necessary experimenter inputs appropriately.

INPUT VARIABLES:
block_id:          [String (BlockX, where X is an int))]; Block ID of the task that was run.
date:              [string (YYYY_MM_DD)]; Date on which the block was run.
dir_bci2000_data:  [string]; Directory where the BCI2000 data is stored.
patient_id:        [string]; Patient ID PYyyNnn or CCxx format, where y, n, and x are integers.
task:              [string]; Type of task that was run.

OUTPUT VARIABLES:
bci2000_data: [dictionary]; Contains all experimentally-related signals, states, and parameters.
"""

# COMPUTATION:

# Loading the .mat file.
bci2000_data = functions_curtailing_bci2000_data.load_bci2000_data(block_id, date, dir_bci2000_data, patient_id, task)


# LOADING TIME LAG BETWEEN BCI2000 AND VIDEO RECORDING

In [4]:
"""
DESCRIPTION:
Loading the time lag between BCI2000 and the video recording. The time lag is imported from:

dir_intermediates + patient_id + '/Speller/BlocksStartAndStops/' + date + '/' + date + '_' + block_id +'_StartStop.txt'

Feel free to modify the pathway in which these starting and ending points are stored and the necessary experimenter 
inputs appropriately.

INPUT VARIABLES:
block_id:          [String (BlockX, where X is an int)]; Block ID of the task that was run.
date:              [string (YYYY_MM_DD)]; Date on which the block was run.
dir_intermediates: [string]; [string]; Intermediates directory where relevant information is stored.
patient_id:        [string]; Patient ID PYyyNnn or CCxx format, where y, n, and x are integers.
task:              [string]; Type of task that was run. 

OUTPUT VARIABLES:
t_lag_seconds:  [float (units: s)]; The time lag between the audio from BCI2000 and the video audio. In other words, 
                t_lag is the amount of time that the BCI2000 audio signal is leading the video audio signal. If t_lag =
                150 ms, this means that BCI2000 audio is ahead of the video audio by 150 ms. For example, an audio event
                registered by the video to be at 3.0 s would actually be registered at 3.15 s by BCI2000. 
"""

# COMPUTATION:

# Loading the time lag.
t_lag_seconds = functions_curtailing_bci2000_data.load_time_lag(block_id, date, dir_intermediates, patient_id, task)


TIME LAG (s): 0.247


# LOADING BLOCK START AND STOP TIMES

In [5]:
"""
DESCRIPTION:
Loading the true starting and stopping times for the current block. The starting and stop times are in:

dir_intermediates + patient_id + '/Speller/BlocksStartAndStops/' + date + '/' + date + '_' + block_id +'_StartStop.txt'

Feel free to modify the pathway in which these starting and ending points are stored and the necessary experimenter
inputs appropriately.

INPUT VARIABLES:
block_id:          [String (BlockX, where X is an int))]; Block ID of the task that was run.
date:              [string (YYYY_MM_DD)]; Date on which the block was run.
dir_intermediates: [string]; Intermediates directory where relevant information is stored.
patient_id:        [string]; Patient ID PYyyNnn or CCxx format, where y, n, and x are integers.

OUTPUT VARIABLES:
t_start: [float (units: s)]; True starting time of the block.
t_stop:  [float (units: s)]; True ending time of the block.
"""

# COMPUTATION:

# Loading the starting and stopping times for the block.
t_start,\
t_stop = functions_curtailing_bci2000_data.load_start_stop_times(block_id, date, dir_intermediates, patient_id)


# CREATING TIME ARRAY

In [6]:
"""
DESCRIPTION:
Creating the time array according to the total number of data samples and the sampling rate.

INPUT VARIABLES:
bci2000_data: [dictionary]; Contains all experimentally-related signals, states, and parameters.

OUTPUT VARIABLES:
t_seconds: [array (time samples,) > floats (units: s)]; Time array for the recorded data block.
"""

# COMPUTATION:

# Computing the time array.
t_seconds = functions_curtailing_bci2000_data.creating_time_array(bci2000_data)


# SHIFT AND CURTAIL BCI2000 SIGNALS, STATES, AND TIME ARRAY BY TIME LAG

In [7]:
"""
DESCRIPTION:
Shifting the time array by the time lag and deleting the part of the resulting array that contains negative time values.
Deleting the corresponding parts of the signals array and each array in the states dictionary.

INPUT VARIABLES:
bci2000_data:   [dictionary]; Contains all experimentally-related signals, states, and parameters.
t_lag_seconds:  [float (units: s)]; The time lag between the audio from BCI2000 and the video audio. In other words, 
                t_lag is the amount of time that the BCI2000 audio signal is leading the video audio signal. If t_lag =
                150 ms, this means that BCI2000 audio is ahead of the video audio by 150 ms. For example, an audio event
                registered by the video to be at 3.0 s would actually be registered at 3.15 s by BCI2000. 
t_seconds:      [array (time samples,) > floats (units: s)]; Time array for the recorded data block.

OUTPUT VARIABLES:
signals:     [array (time samples, channels) > floats (units: uV)]; Array of signals for each neural and analog channel
             at each time point. Time samples are curtailed according to the time lag. 
states_dict: [dictionary (Key: string (state); Value: array (time samples,) > ints)]; For each key, there exists an 
             array of integer values describing the state at each time sample. For each array, time samples are 
             curtailed according to the time lag. 
t_seconds:   [array (time samples,) > floats (units: s)]; Time array for the recorded data block. Shifted by the time 
             lag and curtailed such that the resulting negative times are deleted.
"""

# COMPUTATION:

# Curtailing BCI2000 data according to the time lag.
signals,\
states_dict,\
t_seconds = functions_curtailing_bci2000_data.curtail_bci2000_data_by_time_lag(bci2000_data, t_lag_seconds, t_seconds)


# CURTAIL BCI2000 SIGNALS, STATES, AND TIME ARRAY BY BLOCK START AND STOP TIMES

In [8]:
"""
DESCRIPTION:
Curtailing the signals, each array of the states dictionary, and time array such that only the samples 
corresponding to time samples between the block start and stop times remain. 

INPUT VARIABLES:
signals:     [array (time samples, channels) > floats (units: uV)]; Array of signals for each neural and analog channel
             at each time point. Time samples are curtailed according to the time lag. 
states_dict: [dictionary (Key: string (state); Value: array (time samples,) > ints)]; For each key, there exists an 
             array of integer values describing the state at each time sample. For each array, time samples are 
             curtailed according to the time lag. 
t_seconds:   [array (time samples,) > floats (units: s)]; Time array for the recorded data block. Shifted by the time 
             lag and curtailed such that the resulting negative times are deleted.
t_start:     [float (units: s)]; True starting time of the block.
t_stop:      [float (units: s)]; True ending time of the block.

OUTPUT VARIABLES:
signals:     [array (time samples, channels) > floats (units: uV)]; Array of signals for each neural and analog channel
             at each time point. Time samples are curtailed according to the time lag. Time samples are curtailed 
             according to the block start and stop times.
states_dict: [dictionary (Key: string (state); Value: array (time samples,) > ints)]; For each key, there exists an 
             array of integer values describing the state at each time sample. For each array, time samples are 
             curtailed according to the block start and stop times. 
t_seconds:   [array (time samples,) > floats (units: s)]; Time array for the recorded data block. Time samples are
             curtailed according to the block start and stop times.
"""

# COMPUTATION:

# Curtaling the BCI2000 data according to the block start and stop times.
signals,\
states_dict,\
t_seconds = functions_curtailing_bci2000_data.curtail_bci2000_data_by_block_start_stop_times(signals, states_dict,\
                                                                                             t_seconds, t_start, t_stop)


# SAVING MODIFIED SIGNALS AND STATES

In [9]:
"""
DESCRIPTION:
Overwriting the signals and states values in the BCI2000 data dictionary. Also adding a 'time' key to which holds the 
time array. Saving this BCI2000 data as a new .mat file in the following pathway:

dir_bci2000_data + patient_id + '/mat/' + date + '/' + task + '_Adjusted_' + block_id + '.mat'

Feel free to modify the pathway in which this .mat file is saved and the necessary experimenter inputs appropriately.

INPUT VARIABLES:
bci2000_data:     [dictionary]; Contains all experimentally-related signals, states, and parameters.
block_id:         [String (BlockX, where X is an int))]; Block ID of the task that was run.
date:             [string (YYYY_MM_DD)]; Date on which the block was run.
dir_bci2000_data: [string]; Directory where the BCI2000 data is stored.
patient_id:       [string]; Patient ID PYyyNnn or CCxx format, where y, n, and x are integers.
signals:          [array (time samples, channels) > floats (units: uV)]; Array of signals for each neural and analog 
                  channel at each time point. Time samples are curtailed according to the time lag. Time samples are 
                  curtailed according to the block start and stop times.
states_dict:      [dictionary (Key: string (state); Value: array (time samples,) > ints)]; For each key, there exists an 
                  array of integer values describing the state at each time sample. For each array, time samples are 
                  curtailed according to the block start and stop times. 
t_seconds:        [array (time samples,) > floats (units: s)]; Time array for the recorded data block. Time samples are
                  curtailed according to the block start and stop times.
task:             [string]; Type of task that was run.
"""

# COMPUTATION:

# Saving the curtailed BCI2000 data.
functions_curtailing_bci2000_data.save_curtailed_BCI2000_data(bci2000_data, block_id, date, dir_bci2000_data,\
                                                              patient_id, signals, states_dict, t_seconds, task)
